In [1]:
import os
import pandas as pd

# Mostrar configuración general
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)


In [2]:
# Rutas base del proyecto
DATA_RAW = "../data/external"
DATA_INTERIM = "../data/interim"
DATA_PROCESSED = "../data/processed"

# Crear carpetas si no existen
os.makedirs(DATA_INTERIM, exist_ok=True)
os.makedirs(DATA_PROCESSED, exist_ok=True)

print("📁 Carpetas listas:")
print(f"RAW: {DATA_RAW}")
print(f"INTERIM: {DATA_INTERIM}")
print(f"PROCESSED: {DATA_PROCESSED}")


📁 Carpetas listas:
RAW: ../data/external
INTERIM: ../data/interim
PROCESSED: ../data/processed


In [3]:
archivos = [
    "estatal_itaee.csv",
    "estatal_pea.csv",
    "estatal_poblacion_desocupada.csv",
    "estatal_rezago_educativo.csv",
    "estatal_sector_informal.csv",
    "grado_promedio_escolaridad.csv",
    "nacional_gasto_educacion_pib.csv",
    "poblacion_asiste_escuela.csv",
    "estatal_con_educacion_basica.csv",
    "estatal_ingresos_hasta_1_salmin.csv",
    "poblacion_total.csv",
    "porcentaje_alfabetas.csv",
    "total_viviendas_habitadas.csv",
    "viviendas_con_automovil.csv",
    "viviendas_con_internet.csv"
]

print(f"Total de archivos a procesar: {len(archivos)}")


Total de archivos a procesar: 15


In [4]:
def procesar_archivo(nombre_archivo):
    """
    Lee, limpia y estructura un archivo CSV para convertirlo a formato tidy.
    """
    ruta = os.path.join(DATA_RAW, nombre_archivo)
    print(f"📂 Procesando: {nombre_archivo}")
    
    # Leer CSV
    try:
        df = pd.read_csv(ruta)
    except Exception as e:
        print(f"⚠️ Error al leer {nombre_archivo}: {e}")
        return None

    # Estandarizar nombres de columnas
    df.columns = df.columns.str.lower().str.strip()
    
    # Verificar si tiene 'municipio'
    if 'municipio' in df.columns:
        tidy = df[['municipio', 'periodo', 'valor']].copy()
    else:
        tidy = df[['periodo', 'valor']].copy()
        tidy['municipio'] = None
    
    # Agregar metadatos
    tidy['fuente'] = nombre_archivo.replace(".csv", "")
    tidy['nivel'] = 'estatal' if 'estatal' in nombre_archivo else 'nacional'
    
    # Asegurar tipos correctos
    tidy['periodo'] = tidy['periodo'].astype(str)
    tidy['valor'] = pd.to_numeric(tidy['valor'], errors='coerce')
    
    # Guardar versión intermedia
    nombre_salida = nombre_archivo.replace(".csv", "_tidy.csv")
    tidy.to_csv(os.path.join(DATA_INTERIM, nombre_salida), index=False)
    
    return tidy


In [5]:
tidy_data = []

for archivo in archivos:
    df_tidy = procesar_archivo(archivo)
    if df_tidy is not None:
        tidy_data.append(df_tidy)

# Concatenar todos los datos
tidy_df = pd.concat(tidy_data, ignore_index=True)
print(f"✅ Total de filas combinadas: {len(tidy_df)}")


📂 Procesando: estatal_itaee.csv
📂 Procesando: estatal_pea.csv
📂 Procesando: estatal_poblacion_desocupada.csv
📂 Procesando: estatal_rezago_educativo.csv
📂 Procesando: estatal_sector_informal.csv
📂 Procesando: grado_promedio_escolaridad.csv
📂 Procesando: nacional_gasto_educacion_pib.csv
📂 Procesando: poblacion_asiste_escuela.csv
📂 Procesando: estatal_con_educacion_basica.csv
📂 Procesando: estatal_ingresos_hasta_1_salmin.csv
📂 Procesando: poblacion_total.csv
📂 Procesando: porcentaje_alfabetas.csv
📂 Procesando: total_viviendas_habitadas.csv
📂 Procesando: viviendas_con_automovil.csv
📂 Procesando: viviendas_con_internet.csv
✅ Total de filas combinadas: 2655


In [ ]:
# Guardar dataset combinado en data/processed/
output_path = os.path.join(DATA_PROCESSED, "sonora_educacion_tidy_inegi.csv")
tidy_df.to_csv(output_path, index=False)

print(f"✅ Dataset final guardado en:\n{output_path}")
tidy_df.sample(10)


✅ Dataset final guardado en:
../data/processed\sonora_educacion_tidy.csv


,periodo,valor,municipio,fuente,nivel
1805,2015,96.920290,Divisaderos,porcentaje_alfabetas,nacional
180,1980/01,38.557052,None,estatal_itaee,estatal
...,...,...,...,...,...
2457,2020,60.619469,La Colorada,viviendas_con_automovil,nacional
2545,2015,38.104133,Cajeme,viviendas_con_internet,nacional


In [12]:
print("🔍 Vista general de los datos:")
display(tidy_df.tail(20))

print("\nColumnas disponibles:")
print(list(tidy_df.columns))

print("\nNúmero de fuentes distintas:", tidy_df['fuente'].nunique())
print("Ejemplo de fuentes:", tidy_df['fuente'].unique()[:5])


🔍 Vista general de los datos:


,periodo,valor,municipio,fuente,nivel
2635,2015,26.485149,Tepache,viviendas_con_internet,nacional
2636,2020,51.385390,Tepache,viviendas_con_internet,nacional
...,...,...,...,...,...
2653,2015,11.702688,San Ignacio Rio Muerto,viviendas_con_internet,nacional
2654,2020,21.044547,San Ignacio Rio Muerto,viviendas_con_internet,nacional



Columnas disponibles:
['periodo', 'valor', 'municipio', 'fuente', 'nivel']

Número de fuentes distintas: 15
Ejemplo de fuentes: ['estatal_itaee' 'estatal_pea' 'estatal_poblacion_desocupada'
 'estatal_rezago_educativo' 'estatal_sector_informal']
